This notebook runs through the process of using the `vanna` Python package to generate SQL using AI (RAG + LLMs) including connecting to a database and training. If you're not ready to train on your own database, you can still try it using a sample [SQLite database](app.md).

## Setup

In [ ]:
#%pip install 'vanna[chromadb,openai,postgres]'

In [ ]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore


In [ ]:
from vanna.remote import VannaDefault
vn = VannaDefault(model='analitica_portal_amb_cartografia', api_key='83c53d8e21cd431cb1e178123bca535f')




In [ ]:
vn.connect_to_postgres(host='aws-0-eu-central-1.pooler.supabase.com',
                                  dbname='postgres',
                                  user='postgres.spdwbcfeoefxnlfdhlgi',
                                  password='chatbot2025',
                                  port='6543')

## Training
Solo se necesita hacer una vez.
si se corre varias veces generar nuevamente vectores inecesarios.

In [ ]:

vn.train(ddl='''-- Crear tabla click
CREATE TABLE IF NOT EXISTS public.click (
    epsg25831_x DOUBLE PRECISION,
    epsg25831_y DOUBLE PRECISION,
    id INTEGER PRIMARY KEY,
    idcategoria VARCHAR,
    idproducto INTEGER,
    usuario VARCHAR,
    fecha TIMESTAMP WITHOUT TIME ZONE,
    nivelzoom INTEGER,
    ididioma VARCHAR,
    nomproducte VARCHAR,
    idioma VARCHAR,
    lon DOUBLE PRECISION,
    lat DOUBLE PRECISION
);

-- Crear tabla descargas
CREATE TABLE IF NOT EXISTS public.descargas (
    id INTEGER PRIMARY KEY,
    idfull VARCHAR,
    idcategoria VARCHAR,
    fechadescarga TIMESTAMP WITHOUT TIME ZONE,
    idepoca VARCHAR,
    idproducto INTEGER,
    mensaje VARCHAR,
    nombrefichero VARCHAR,
    usuario VARCHAR,
    nomproducte VARCHAR,
    format VARCHAR,
    idepoca VARCHAR
);

-- Crear tabla usuarios
CREATE TABLE IF NOT EXISTS public.usuarios (
    usuario VARCHAR PRIMARY KEY,
    fechaacepto TIMESTAMP WITHOUT TIME ZONE,
    fechaalta TIMESTAMP WITHOUT TIME ZONE,
    genero VARCHAR,
    ididioma VARCHAR,
    notificaciones BOOLEAN,
    procedencia VARCHAR,
    idioma VARCHAR,
    idperfil INTEGER,
    nomperfil VARCHAR,
    idambito INTEGER,
    nomambito VARCHAR ,
    idconfirmado INTEGER,
    confirmat VARCHAR,
    correo VARCHAR,
    nombre VARCHAR
);

-- Crear tabla fulls --
CREATE IF NOT EXISTS public.TABLE fulls (
    idfull SERIAL PRIMARY KEY,
    nomproducto TEXT NOT NULL,
    geometria GEOMETRY NOT NULL
);

-- Crear claves foráneas para la integridad referencial
ALTER TABLE public.click
    ADD CONSTRAINT fk_click_usuario FOREIGN KEY (usuario) REFERENCES public.usuarios(usuario),
    ADD CONSTRAINT fk_click_idcategoria FOREIGN KEY (idcategoria) REFERENCES public.descargas(idcategoria),
    ADD CONSTRAINT fk_click_nomproducte FOREIGN KEY (nomproducte) REFERENCES public.descargas(nomproducte);

ALTER TABLE public.descargas
    ADD CONSTRAINT fk_descargas_usuario FOREIGN KEY (usuario) REFERENCES public.usuarios(usuario),


-- Crear clave foránea para relacionar fulls con descargas mediante idfull
ALTER TABLE public.descargas
    ADD CONSTRAINT fk_descargas_idfull FOREIGN KEY (idfull) REFERENCES public.fulls(idfull);

'''

)

Adding ddl: -- Crear tabla click
CREATE TABLE IF NOT EXISTS public.click (
    epsg25831_x DOUBLE PRECISION,
    epsg25831_y DOUBLE PRECISION,
    id INTEGER PRIMARY KEY,
    idcategoria VARCHAR,
    idproducto INTEGER,
    usuario VARCHAR,
    fecha TIMESTAMP WITHOUT TIME ZONE,
    nivelzoom INTEGER,
    ididioma VARCHAR,
    nomproducte VARCHAR,
    idioma VARCHAR,
    lon DOUBLE PRECISION,
    lat DOUBLE PRECISION
);

-- Crear tabla descargas
CREATE TABLE IF NOT EXISTS public.descargas (
    id INTEGER PRIMARY KEY,
    idfull VARCHAR,
    idcategoria VARCHAR,
    fechadescarga TIMESTAMP WITHOUT TIME ZONE,
    idepoca VARCHAR,
    idproducto INTEGER,
    mensaje VARCHAR,
    nombrefichero VARCHAR,
    usuario VARCHAR,
    nomproducte VARCHAR,
    format VARCHAR,
    idepoca VARCHAR
);

-- Crear tabla usuarios
CREATE TABLE IF NOT EXISTS public.usuarios (
    usuario VARCHAR PRIMARY KEY,
    fechaacepto TIMESTAMP WITHOUT TIME ZONE,
    fechaalta TIMESTAMP WITHOUT TIME ZONE,
    genero 

'1249795-ddl'

In [ ]:
vn.train(
    question="Indica les descàrregues totals de tots els productes que distribuïm",
    sql="""
        SELECT COUNT(*) AS total_descargas
        FROM public.descargas;
    """
)

vn.train(
    question="Indica les descàrregues totals de tots els productes que distribuïm per producte",
    sql="""
        SELECT nomproducte, COUNT(*) AS total_descargas
        FROM public.descargas
        GROUP BY nomproducte
        ORDER BY total_descargas DESC;
    """
)

vn.train(
    question="Indica les descàrregues totals de tots els productes que distribuïm per categoria",
    sql="""
        SELECT idcategoria, COUNT(*) AS total_descargas
        FROM public.descargas
        GROUP BY idcategoria
        ORDER BY total_descargas DESC;
    """
)

vn.train(
    question="Quin és el número de descàrregues totals durant l’any 2024?",
    sql="""
        SELECT COUNT(*) AS total_descargas_2024
        FROM public.descargas
        WHERE EXTRACT(YEAR FROM fechadescarga) = 2024;
    """
)

'd77cef4b7805e7865d88480352f8fdbc-sql'

In [ ]:


vn.train(
    question="Quin és el número de descàrregues totals durant el segon semestre de l’any 2021 per producte.",
    sql="""
        SELECT nomproducte, COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE fechadescarga BETWEEN '2021-07-01' AND '2021-12-31'
        GROUP BY nomproducte
        ORDER BY total_descargas DESC;
    """
)

vn.train(
    question="Quin és el número de descàrregues totals durant quart trimestre de l’any 2020",
    sql="""
        SELECT COUNT(*) AS total_descargas_q4_2020
        FROM public.descargas
        WHERE fechadescarga BETWEEN '2020-10-01' AND '2020-12-31';
    """
)

vn.train(
    question="Crea un gràfic de línies on visualitzem l’evolució de les descàrregues per any",
    sql="""
        SELECT EXTRACT(YEAR FROM fechadescarga) AS y, COUNT(*) AS total_descargas
        FROM public.descargas
        GROUP BY y
        ORDER BY y;
    """
)

vn.train(
    question="Crea un gràfic de línies on visualitzem l’evolució de les descàrregues per mes de Cartografia topobatimètric",
    sql="""
        SELECT
            DATE_TRUNC('month', fechadescarga) AS mes,
            COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE nomproducte ILIKE '%topobatim%'
        GROUP BY mes
        ORDER BY mes;
    """
)

vn.train(
    question="Crea un gràfic de línies on visualitzem l’evolució de les descàrregues per mes de Models digitals històrics",
    sql="""
        SELECT
            DATE_TRUNC('month', fechadescarga) AS mes,
            COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE idcategoria ILIKE '%digitalshist%'
        GROUP BY mes
        ORDER BY mes;
    """
)

vn.train(
    question="Mostra’m en un gràfic de barres verticals de les descàrregues anuals.",
    sql="""
        SELECT EXTRACT(YEAR FROM fechadescarga) AS y, COUNT(*) AS total_descargas
        FROM public.descargas
        GROUP BY y
        ORDER BY y;
    """
)

vn.train(
    question="Generem una taula amb les descàrregues totals i de les parcials separades per categories i productes de l’any 2020 ",
    sql="""
        WITH descargas_producto AS (
            SELECT
                idcategoria,
                nomproducte,
                COUNT(*) AS total_descargas_producto
            FROM public.descargas
            WHERE EXTRACT(YEAR FROM fechadescarga) = 2020
            GROUP BY idcategoria, nomproducte
        ),
        descargas_categoria AS (
            SELECT
                idcategoria,
                SUM(total_descargas_producto) AS total_descargas_categoria
            FROM descargas_producto
            GROUP BY idcategoria
        )
        SELECT
            dp.idcategoria,
            dp.nomproducte,
            dp.total_descargas_producto AS descargas_parciales,
            dc.total_descargas_categoria AS descargas_totales_categoria
        FROM descargas_producto dp
        JOIN descargas_categoria dc ON dp.idcategoria = dc.idcategoria
        ORDER BY dc.total_descargas_categoria DESC, dp.total_descargas_producto DESC;
    """
)

'148fec22069a59d73af6ac6c9ff6fea6-sql'

In [ ]:


vn.train(
    question="Mostra’m una taula amb la classificació de les 5 productes més descarregudes",
    sql="""
        WITH descargas_producto AS (
            SELECT
                idcategoria,
                nomproducte,
                COUNT(*) AS total_descargas_producto
            FROM public.descargas
            WHERE EXTRACT(YEAR FROM fechadescarga) = 2020
            GROUP BY idcategoria, nomproducte
        ),
        descargas_categoria AS (
            SELECT
                idcategoria,
                SUM(total_descargas_producto) AS total_descargas_categoria
            FROM descargas_producto
            GROUP BY idcategoria
        )
        SELECT
            dp.idcategoria,
            dp.nomproducte,
            dp.total_descargas_producto AS descargas_parciales,
            dc.total_descargas_categoria AS descargas_totales_categoria
        FROM descargas_producto dp
        JOIN descargas_categoria dc ON dp.idcategoria = dc.idcategoria
        ORDER BY dc.total_descargas_categoria DESC, dp.total_descargas_producto DESC;
    """
)

vn.train(
    question="Quina es la categoria que té mes descàrregues en el primer trimestre de l’any  2023",
    sql="""
        SELECT idcategoria, COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE fechadescarga BETWEEN '2023-01-01' AND '2023-03-31'
        GROUP BY idcategoria
        ORDER BY total_descargas DESC
        LIMIT 1;
    """
)

vn.train(
    question="Fes un gràfic de línia mes a mes que compari els 5 productes més descarregats.",
    sql="""
        WITH top_productes AS (
            SELECT nomproducte
            FROM public.descargas
            GROUP BY nomproducte
            ORDER BY COUNT(*) DESC
            LIMIT 5
        )
        SELECT
            DATE_TRUNC('month', fechadescarga) AS mes,
            nomproducte,
            COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE nomproducte IN (SELECT nomproducte FROM top_productes)
        GROUP BY mes, nomproducte
        ORDER BY mes, total_descargas DESC;
    """
)

vn.train(
    question="En quin moment de l’any s’han realitzat més descàrregues?",
    sql="""
        WITH descargas_mensuales AS (
            SELECT
                TO_CHAR(fechadescarga, 'Month') AS mes,
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            GROUP BY y, mes
        )
        SELECT
            TRIM(mes) AS mes,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_mensuales
        GROUP BY mes
        ORDER BY promedio_descargas DESC
        LIMIT 1;
    """
)

vn.train(
    question="Genera un gràfic on es visualitzin les descàrregues mes a mes.",
    sql="""
        WITH descargas_mensuales AS (
            SELECT
                EXTRACT(MONTH FROM fechadescarga) AS numero_mes,
                TO_CHAR(fechadescarga, 'Month') AS mes,
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            GROUP BY numero_mes, mes, y
        )
        SELECT
            numero_mes,
            TRIM(mes) AS mes,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_mensuales
        GROUP BY numero_mes, mes
        ORDER BY numero_mes;
    """
)

vn.train(
    question="Quin mes es produeixen més descàrregues d'ortofotos de platges metropolitanes?",
    sql="""
        SELECT
            TO_CHAR(fechadescarga, 'Month') AS mes,
            COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE idcategoria ILIKE '%platg%'
        GROUP BY mes
        ORDER BY total_descargas DESC
        LIMIT 1;
    """
)

vn.train(
    question="Genera un gràfic de descàrregues per mes de productes de platges.",
    sql="""
        WITH descargas_mensuales AS (
            SELECT
                EXTRACT(MONTH FROM fechadescarga) AS numero_mes,
                TO_CHAR(fechadescarga, 'Month') AS mes,
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            WHERE idcategoria ILIKE '%platg%'
            GROUP BY numero_mes, mes, y
        )
        SELECT
            numero_mes,
            TRIM(mes) AS mes,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_mensuales
        GROUP BY numero_mes, mes
        ORDER BY numero_mes;
    """
)

vn.train(
    question="Quina és la categoria que té més descàrregues durant l’any 2020?",
    sql="""
        SELECT idcategoria, COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE EXTRACT(YEAR FROM fechadescarga) = 2020
        GROUP BY idcategoria
        ORDER BY total_descargas DESC
        LIMIT 1;
    """
)

vn.train(
    question="En quin moment de l’any actual s’han realitzat més descàrregues?",
    sql="""
        SELECT
            TO_CHAR(fechadescarga, 'Month') AS mes,
            COUNT(*) AS total_descargas
        FROM public.descargas
        WHERE EXTRACT(YEAR FROM fechadescarga) = EXTRACT(YEAR FROM CURRENT_DATE)
        GROUP BY mes
        ORDER BY total_descargas DESC
        LIMIT 1;
    """
)

vn.train(
    question="Diguem si durant l’any 2024 hi ha hagut més o menys descàrregues respecte l’any anterior.",
    sql="""
        WITH descargas_por_año AS (
            SELECT
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            WHERE EXTRACT(YEAR FROM fechadescarga) IN (2023, 2024)
            GROUP BY y
        )
        SELECT
            MAX(CASE WHEN y = 2024 THEN total_descargas END) AS descargas_2024,
            MAX(CASE WHEN y = 2023 THEN total_descargas END) AS descargas_2023,
            CASE
                WHEN MAX(CASE WHEN y = 2024 THEN total_descargas END) > MAX(CASE WHEN y = 2023 THEN total_descargas END)
                THEN 'MÉS DESCÀRREGUES EL 2024'
                ELSE 'MENYS DESCÀRREGUES EL 2024'
            END AS comparació
        FROM descargas_por_año;
    """
)


'9638d63f942777040d2c2aa6d3736b6a-sql'

In [ ]:

vn.train(
    question="En quins mesos es produeixen més descàrregues?",
    sql="""
        WITH descargas_mensuales AS (
            SELECT
                EXTRACT(MONTH FROM fechadescarga) AS numero_mes,
                TO_CHAR(fechadescarga, 'Month') AS mes,
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            GROUP BY numero_mes, mes, y
        )
        SELECT
            numero_mes,
            TRIM(mes) AS mes,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_mensuales
        GROUP BY numero_mes, mes
        ORDER BY numero_mes;
    """
)

vn.train(
    question="Quin dia de la setmana es produeixen més descàrregues?",
    sql="""
        WITH descargas_diarias AS (
            SELECT
                EXTRACT(DOW FROM fechadescarga) AS dia_setmana,
                COUNT(*) AS total_descargas
            FROM public.descargas
            GROUP BY dia_setmana, EXTRACT(YEAR FROM fechadescarga)
        )
        SELECT
            CASE
                WHEN dia_setmana IN (1, 2, 3) THEN 'Inici de setmana (dilluns-dimecres)'
                WHEN dia_setmana IN (4, 5, 6, 0) THEN 'Dijous-diumenge'
            END AS periode,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_diarias
        GROUP BY periode
        ORDER BY promedio_descargas DESC;
    """
)

vn.train(
    question="En quin període de l’any augmenten les descàrregues de la categoria vols històrics?",
    sql="""
        WITH descargas_mensuales AS (
            SELECT
                EXTRACT(MONTH FROM fechadescarga) AS numero_mes,
                TO_CHAR(fechadescarga, 'Month') AS mes,
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            WHERE idcategoria ILIKE '%vols%hist%'
            GROUP BY numero_mes, mes, y
        )
        SELECT
            numero_mes,
            TRIM(mes) AS mes,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_mensuales
        GROUP BY numero_mes, mes
        ORDER BY numero_mes;
    """
)

vn.train(
    question="Quan es fan més descàrregues, a l’inici, a meitat o al final de cada mes?",
    sql="""
        WITH descargas_periodo AS (
            SELECT
                CASE
                    WHEN EXTRACT(DAY FROM fechadescarga) <= 10 THEN 'Inici del mes'
                    WHEN EXTRACT(DAY FROM fechadescarga) <= 20 THEN 'Meitat del mes'
                    ELSE 'Final del mes'
                END AS periode,
                EXTRACT(YEAR FROM fechadescarga) AS y,
                COUNT(*) AS total_descargas
            FROM public.descargas
            GROUP BY periode, y
        )
        SELECT
            periode,
            ROUND(AVG(total_descargas), 2) AS promedio_descargas
        FROM descargas_periodo
        GROUP BY periode
        ORDER BY promedio_descargas DESC;
    """
)


'11fd7c7a6f134851d1c8a704aebe1686-sql'

In [ ]:
vn.train(question="Identifica’m patrons de descàrregues que es produeixin setmanalment o mensualment.",
    sql="""
        WITH descargas_semanales AS (
    SELECT
        EXTRACT(DOW FROM fechadescarga) AS numero_dia,
        TO_CHAR(fechadescarga, 'Day') AS dia,
        EXTRACT(YEAR FROM fechadescarga) AS y,
        COUNT(*) AS total_descargas
    FROM public.descargas
    GROUP BY numero_dia, dia, y
),
descargas_mensuales AS (
    SELECT
        EXTRACT(MONTH FROM fechadescarga) AS numero_mes,
        TO_CHAR(fechadescarga, 'Month') AS mes,
        EXTRACT(YEAR FROM fechadescarga) AS y,
        COUNT(*) AS total_descargas
    FROM public.descargas
    GROUP BY numero_mes, mes, y
),
descargas_periodo AS (
    SELECT
        CASE
            WHEN EXTRACT(DAY FROM fechadescarga) <= 10 THEN 'Inici del mes'
            WHEN EXTRACT(DAY FROM fechadescarga) <= 20 THEN 'Meitat del mes'
            ELSE 'Final del mes'
        END AS periode,
        EXTRACT(YEAR FROM fechadescarga) AS y,
        COUNT(*) AS total_descargas
    FROM public.descargas
    GROUP BY periode, y
)
SELECT
    'Setmanal' AS tipus_analisi,
    numero_dia AS identificador,
    TRIM(dia) AS nom,
    ROUND(AVG(total_descargas), 2) AS promedio_descargas
FROM descargas_semanales
GROUP BY numero_dia, nom
UNION ALL
SELECT
    'Mensual' AS tipus_analisi,
    numero_mes AS identificador,
    TRIM(mes) AS nom,
    ROUND(AVG(total_descargas), 2) AS promedio_descargas
FROM descargas_mensuales
GROUP BY numero_mes, nom
UNION ALL
SELECT
    'Periode Mes' AS tipus_analisi,
    NULL AS identificador,
    periode AS nom,
    ROUND(AVG(total_descargas), 2) AS promedio_descargas
FROM descargas_periodo
GROUP BY periode
ORDER BY tipus_analisi, identificador NULLS LAST;
        """
)

'125dada33506159e59ae59edd3639979-sql'

In [ ]:

vn.train(
    question="Que tant varia la descàrrega de dades LiDAR abans i després del 5 de maig de 2024?",
    sql="""
        WITH descargas_30_15_antes AS (
            SELECT COUNT(*) AS total_30_15_antes
            FROM public.descargas
            WHERE idcategoria ILIKE '%lidar%'
            AND fechadescarga BETWEEN '2024-04-05' AND '2024-04-19'
        ),
        descargas_15_antes AS (
            SELECT COUNT(*) AS total_15_antes
            FROM public.descargas
            WHERE idcategoria ILIKE '%lidar%'
            AND fechadescarga BETWEEN '2024-04-20' AND '2024-05-04'
        ),
        descargas_15_despues AS (
            SELECT COUNT(*) AS total_15_despues
            FROM public.descargas
            WHERE idcategoria ILIKE '%lidar%'
            AND fechadescarga BETWEEN '2024-05-05' AND '2024-05-19'
        ),
        descargas_30_despues AS (
            SELECT COUNT(*) AS total_30_despues
            FROM public.descargas
            WHERE idcategoria ILIKE '%lidar%'
            AND fechadescarga BETWEEN '2024-05-20' AND '2024-06-04'
        )
        SELECT
            total_30_15_antes,
            total_15_antes,
            total_15_despues,
            total_30_despues
        FROM descargas_30_15_antes, descargas_15_antes, descargas_15_despues, descargas_30_despues;
        """
)

vn.train(
    question="Quants usuaris nous es registren cada mes?",
    sql="""
        WITH registros_mensuales AS (
            SELECT
                EXTRACT(MONTH FROM fechaalta) AS numero_mes,
                TO_CHAR(fechaalta, 'Month') AS mes,
                EXTRACT(YEAR FROM fechaalta) AS y,
                COUNT(*) AS total_registros
            FROM public.usuarios
            GROUP BY numero_mes, mes, y
        )
        SELECT
            numero_mes,
            TRIM(mes) AS mes,
            ROUND(AVG(total_registros), 2) AS promedio_registros
        FROM registros_mensuales
        GROUP BY numero_mes, mes
        ORDER BY numero_mes;
        """
)

'c56d9bf785984d2fbc3e56a1f8e8674b-sql'

In [ ]:
vn.train(
    question="Quants usuaris han descarregat dades almenys una vegada?",
    sql="""
        SELECT COUNT(DISTINCT usuario) AS usuarios_con_descargas
        FROM public.descargas;
        """
)

vn.train(
    question="Quants usuaris estan inactius?",
    sql="""
        SELECT COUNT(*) AS usuarios_inactivos
        FROM public.usuarios
        WHERE fechaacepto IS NULL;
        """
)

vn.train(
    question="Quant temps passa en mitjana entre el registre i la primera descàrrega?",
    sql="""
        WITH primera_descarga AS (
            SELECT usuario, MIN(fechadescarga) AS primera_descarga
            FROM public.descargas
            GROUP BY usuario
        )
        SELECT
            ROUND(AVG(EXTRACT(EPOCH FROM AGE(pd.primera_descarga, u.fechaalta)) / 86400), 2) AS tiempo_promedio_dias
        FROM primera_descarga pd
        JOIN public.usuarios u ON pd.usuario = u.usuario
        WHERE pd.primera_descarga IS NOT NULL;
        """
)

vn.train(
    question="Quants usuaris no han fet cap descàrrega?",
    sql="""
    WITH total_usuarios AS (
        SELECT COUNT(*) AS total FROM public.usuarios
    ),
    usuarios_sin_descargas AS (
        SELECT COUNT(*) AS sin_descargas
        FROM public.usuarios u
        LEFT JOIN public.descargas d ON u.usuario = d.usuario
        WHERE d.usuario IS NULL
    )
    SELECT
        usd.sin_descargas AS usuarios_sin_descargas,
        tu.total AS total_usuarios,
        ROUND((usd.sin_descargas * 100.0 / tu.total), 2) AS porcentaje_usuarios_sin_descargas
    FROM usuarios_sin_descargas usd, total_usuarios tu;
    """
)

vn.train(
    question="Quants usuaris no han fet cap descàrrega per procedència?",
    sql="""
    WITH total_usuarios AS (
        SELECT COUNT(*) AS total FROM public.usuarios
    ),
    usuarios_sin_descargas AS (
        SELECT u.procedencia, COUNT(*) AS sin_descargas
        FROM public.usuarios u
        LEFT JOIN public.descargas d ON u.usuario = d.usuario
        WHERE d.usuario IS NULL
        GROUP BY u.procedencia
    )
    SELECT
        usd.procedencia,
        usd.sin_descargas AS usuarios_sin_descargas,
        tu.total AS total_usuarios,
        ROUND((usd.sin_descargas * 100.0 / tu.total), 2) AS porcentaje_sin_descargas
    FROM usuarios_sin_descargas usd, total_usuarios tu
    ORDER BY porcentaje_sin_descargas DESC;
    """
)

vn.train(
    question="Quin és el perfil professional més comú entre els usuaris?",
    sql="""
    SELECT
        nomperfil,
        COUNT(*) AS total_usuarios
    FROM public.usuarios
    GROUP BY nomperfil
    ORDER BY total_usuarios DESC;
    """
)

vn.train(
    question="Com ha evolucionat la distribució de perfils professionals amb el temps?",
    sql="""
    WITH registros_perfil AS (
        SELECT
            EXTRACT(YEAR FROM fechaalta) AS y,
            EXTRACT(MONTH FROM fechaalta) AS numero_mes,
            TO_CHAR(fechaalta, 'Month') AS mes,
            nomperfil,
            COUNT(*) AS total_registros
        FROM public.usuarios
        GROUP BY y, numero_mes, mes, nomperfil
    )
    SELECT
        y AS any,
        numero_mes,
        TRIM(mes) AS mes,
        nomperfil,
        total_registros
    FROM registros_perfil
    ORDER BY y, numero_mes, total_registros DESC;
    """
)

vn.train(
    question="Quins són els àmbits professionals amb més usuaris registrats?",
    sql="""
    SELECT
        nomambito,
        COUNT(*) AS total_usuarios
    FROM public.usuarios
    GROUP BY nomambito
    ORDER BY total_usuarios DESC;
    """
)

vn.train(
    question="Quins són els 5 àmbits professionals principals que més descarreguen les xarxes geodèsiques?",
    sql="""
    SELECT
        u.nomambito,
        COUNT(*) AS total_descargas
    FROM public.descargas d
    JOIN public.usuarios u ON d.usuario = u.usuario
    WHERE d.idcategoria ILIKE '%geod%'
    GROUP BY u.nomambito
    ORDER BY total_descargas DESC
    LIMIT 5;
    """
)

vn.train(
    question="Gràfica de cercle dels usuaris per àmbit professional?",
    sql="""
    WITH total AS (
        SELECT COUNT(*) AS total_usuarios FROM public.usuarios
    )
    SELECT
        u.nomambito,
        COUNT(*) AS total_usuarios,
        ROUND((COUNT(*) * 100.0 / t.total_usuarios), 2) AS porcentaje
    FROM public.usuarios u, total t
    GROUP BY u.nomambito, t.total_usuarios
    ORDER BY total_usuarios DESC;
    """
)

vn.train(
    question="Gràfica de cercle dels usuaris per àmbit professional que descarreguen ortofoto 1965 pel que fa als qui descarrega ortofoto 2020?",
    sql="""
    WITH usuarios_1965 AS (
        SELECT u.nomambito, COUNT(*) AS total_usuarios
        FROM public.descargas d
        JOIN public.usuarios u ON d.usuario = u.usuario
        WHERE d.nomproducte ILIKE '%orto%1965%'
        GROUP BY u.nomambito
    ),
    usuarios_2020 AS (
        SELECT u.nomambito, COUNT(*) AS total_usuarios
        FROM public.descargas d
        JOIN public.usuarios u ON d.usuario = u.usuario
        WHERE d.nomproducte ILIKE '%orto%2020%'
        GROUP BY u.nomambito
    )
    SELECT
        COALESCE(u1965.nomambito, u2020.nomambito) AS nomambito,
        COALESCE(u1965.total_usuarios, 0) AS usuaris_1965,
        COALESCE(u2020.total_usuarios, 0) AS usuaris_2020
    FROM usuarios_1965 u1965
    FULL JOIN usuarios_2020 u2020 ON u1965.nomambito = u2020.nomambito
    ORDER BY usuaris_1965 DESC, usuaris_2020 DESC;
    """
)

'37aa8d474ca940384134bbbd323114c2-sql'

In [ ]:

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'public' AND TABLE_NAME NOT IN ('langchain_pg_collection', 'langchain_pg_embedding');")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)



In [ ]:
vn.train(
    question="Quin percentatge d'usuaris registrats pertany a cada idioma?",
    sql="""
    WITH total_usuarios AS (
        SELECT COUNT(*) AS total FROM public.usuarios
    )
    SELECT
        u.idioma AS idioma,
        COUNT(*) AS total_usuarios,
        ROUND((COUNT(*) * 100.0 / t.total), 2) AS percentatge
    FROM public.usuarios u, total_usuarios t
    GROUP BY u.idioma, t.total
    ORDER BY percentatge DESC;
    """
)

vn.train(
    question="Quin percentatge del total d’usuaris representa les dones i els homes?",
    sql="""
    WITH total_filtrado AS (
        SELECT COUNT(*) AS total FROM public.usuarios
        WHERE genero IN ('Hombre', 'Mujer', 'Otros')
    )
    SELECT
        u.genero AS genere,
        COUNT(*) AS total_usuaris,
        ROUND((COUNT(*) * 100.0 / t.total), 2) AS percentatge
    FROM public.usuarios u, total_filtrado t
    WHERE u.genero IN ('Hombre', 'Mujer', 'Otros')
    GROUP BY u.genero, t.total
    ORDER BY percentatge DESC;
    """
)

vn.train(
    question="Com ha canviat els usuaris dones i homes en el temps?",
    sql="""
    WITH registros_genero AS (
        SELECT
            EXTRACT(YEAR FROM fechaalta) AS y,
            EXTRACT(MONTH FROM fechaalta) AS numero_mes,
            TO_CHAR(fechaalta, 'Month') AS mes,
            genero,
            COUNT(*) AS total_usuaris
        FROM public.usuarios
        WHERE genero IN ('Hombre', 'Mujer')
        GROUP BY y, numero_mes, mes, genero
    )
    SELECT
        y, numero_mes, TRIM(mes) AS mes, genero, total_usuaris
    FROM registros_genero
    ORDER BY y, numero_mes, genero;
    """
)

vn.train(
    question="Qui fa més descàrregues, homes o dones?",
    sql="""
    SELECT
        u.genero AS genere,
        COUNT(*) AS total_descargas
    FROM public.descargas d
    JOIN public.usuarios u ON d.usuario = u.usuario
    WHERE u.genero IN ('Hombre', 'Mujer')
    GROUP BY u.genero
    ORDER BY total_descargas DESC;
    """
)

vn.train(
    question="Com varien els patrons de descàrrega segons l'àmbit professional?",
    sql="""
    WITH productos_top AS (
        SELECT
            u.nomambito,
            d.nomproducte,
            COUNT(*) AS total_descargas,
            RANK() OVER (PARTITION BY u.nomambito ORDER BY COUNT(*) DESC) AS ranking
        FROM public.descargas d
        JOIN public.usuarios u ON d.usuario = u.usuario
        GROUP BY u.nomambito, d.nomproducte
    )
    SELECT
        p.nomambito,
        p.nomproducte AS top_producte
    FROM productos_top p
    WHERE p.ranking <= 3
    ORDER BY p.nomambito, p.ranking;
    """
)

vn.train(
    question="Els usuaris que usen un idioma específic són més actius en descàrregues?",
    sql="""
    WITH descargas_por_idioma AS (
        SELECT
            u.idioma AS idioma,
            COUNT(d.id) AS total_descargas,
            COUNT(DISTINCT u.usuario) AS total_usuarios
        FROM public.descargas d
        JOIN public.usuarios u ON d.usuario = u.usuario
        GROUP BY u.idioma
    )
    SELECT
        idioma,
        total_descargas,
        total_usuarios,
        ROUND(total_descargas * 1.0 / total_usuarios, 2) AS promedio_descargas_por_usuario
    FROM descargas_por_idioma
    ORDER BY promedio_descargas_por_usuario DESC;
    """
)

vn.train(
    question="Diguem els 5 usuaris que més descàrregues han realitzat.",
    sql="""
    SELECT
        u.usuario,
        u.correo,
        COUNT(d.id) AS total_descargas
    FROM public.descargas d
    JOIN public.usuarios u ON d.usuario = u.usuario
    GROUP BY u.usuario, u.correo
    ORDER BY total_descargas DESC
    LIMIT 5;
    """
)

vn.train(
    question="Quants usuaris han fet descàrregues de manera constant any rere any?",
    sql="""
    WITH anys_disponibles AS (
        SELECT EXTRACT(YEAR FROM fechadescarga) AS y FROM public.descargas GROUP BY y
    ),
    descargas_por_usuario AS (
        SELECT usuario, EXTRACT(YEAR FROM fechadescarga) AS y FROM public.descargas GROUP BY usuario, y
    ),
    usuarios_constantes AS (
        SELECT d.usuario
        FROM descargas_por_usuario d
        JOIN (SELECT COUNT(*) AS total_anys FROM anys_disponibles) a ON 1=1
        GROUP BY d.usuario, a.total_anys
        HAVING COUNT(DISTINCT d.y) = a.total_anys
    )
    SELECT
        u.usuario,
        u.correo,
        u.nombre
    FROM public.usuarios u
    JOIN usuarios_constantes uc ON u.usuario = uc.usuario
    ORDER BY u.usuario;
    """
)

vn.train(
    question="Quins són els àmbits professionals on les dones realitzen més descàrregues?",
    sql="""
   SELECT
    u.nomambito,
    COUNT(d.id) AS total_descargas_mujer,
    (COUNT(d.id) * 100.0 / NULLIF(td.total_descargas, 0)) AS porcentaje_descargas_mujer
FROM public.descargas d
JOIN public.usuarios u
    ON d.usuario = u.usuario
JOIN
    (
        SELECT
            u.nomambito,
            COUNT(d.id) AS total_descargas
        FROM public.descargas d
        JOIN public.usuarios u
            ON d.usuario = u.usuario
        WHERE u.genero NOT IN ('No respondido', 'No se recoge')
        GROUP BY u.nomambito
    ) td
    ON u.nomambito = td.nomambito
WHERE
    u.genero = 'Mujer'
GROUP BY
    u.nomambito, td.total_descargas
ORDER BY
     porcentaje_descargas_mujer Desc;

    """
)

vn.train(
    question="Existeix alguna relació entre idioma i format de descàrrega preferit?",
    sql="""
    WITH descargas_por_idioma_formato AS (
        SELECT
            u.idioma AS idioma,
            d.format AS formato_descarga,
            COUNT(*) AS total_descargas
        FROM public.descargas d
        JOIN public.usuarios u ON d.usuario = u.usuario
        GROUP BY u.idioma, d.format
    ),
    total_descargas_formato AS (
        SELECT formato_descarga, SUM(total_descargas) AS total_descargas_formato
        FROM descargas_por_idioma_formato
        GROUP BY formato_descarga
    )
    SELECT
        d.formato_descarga,
        d.idioma,
        d.total_descargas,
        t.total_descargas_formato,
        ROUND((d.total_descargas * 100.0 / t.total_descargas_formato), 2) AS porcentaje_idioma_formato
    FROM descargas_por_idioma_formato d
    JOIN total_descargas_formato t ON d.formato_descarga = t.formato_descarga
    ORDER BY d.formato_descarga, porcentaje_idioma_formato DESC;
    """
)

vn.train(
    question="Quins usuaris realitzen descàrregues massives?",
    sql="""
    WITH descargas_marcadas AS (
        SELECT
            usuario,
            fechadescarga,
            COUNT(*) OVER (
                PARTITION BY usuario
                ORDER BY fechadescarga
                RANGE INTERVAL '30 minutes' PRECEDING
            ) AS cantidad_descargas_ventana
        FROM public.descargas
    ),
    eventos_descarga AS (
        SELECT
            usuario,
            fechadescarga
        FROM descargas_marcadas
        WHERE cantidad_descargas_ventana > 50
        GROUP BY usuario, fechadescarga
    ),
    conteo_eventos AS (
        SELECT
            usuario,
            COUNT(*) AS total_eventos
        FROM eventos_descarga
        GROUP BY usuario
    )
    SELECT
        u.usuario,
        u.nombre,
        u.correo,
        ce.total_eventos
    FROM public.usuarios u
    JOIN conteo_eventos ce ON u.usuario = ce.usuario
    ORDER BY ce.total_eventos DESC;
    """
)




'9d6e7c20aaffe369033d2ccfd7faf35b-sql'

In [ ]:
vn.train(
    documentation='''

    "introducción":
        La base de dades del geoportal de cartografia està dissenyada per analitzar la interacció dels usuaris amb el sistema mitjançant dos tipus principals d’accions: clics en productes cartogràfics i descàrregues d’arxius geoespacials, denominats productes. Els productes es descarreguen per zones específiques del territori. Segons les característiques del producte, la totalitat de les àrees del territori disponibles per a descàrrega es subdivideix mitjançant fulls, que són polígons geoespacials.
        Per a això, s’han estructurat quatre taules fonamentals: click, descargas, fulls i usuarios, que es relacionen entre si mitjançant claus foranes.
        '''


)

vn.train(
    documentation='''
        "taula_usuarios":
        La taula usuarios conté la informació sobre cada usuari registrat al sistema. Cada usuari està identificat per un usuario, que actua com a clau primària. A més, es registren els detalls de la seva inscripció, com fechaalta (moment en què l’usuari es va registrar al sistema) i fechaacepto (data en què va acceptar els termes d’ús).
        • nomperfil pot prendre valors com "Públic", "Privat", "Acadèmic" o "Altres".
        • nomambito detalla el sector professional en què opera l’usuari, podent ser "Cartografia i geomàtica", "Medi ambient", "Indústria i energia", entre d’altres.
        • idioma indica la llengua en què l’usuari interactua amb la plataforma.
        • procedencia permet identificar si l’usuari prové de l’àmbit de "CARTOGRAFIA" o "PLANEJAMENT".
      '''

)

vn.train(
    documentation='''

          "taula_click":
        La taula click emmagatzema els esdeveniments en què un usuari interactua amb els productes geoespacials disponibles al geoportal.
        • fecha indica el moment exacte en què va ocórrer la interacció.
        • idproducto i nomproducte permeten identificar el producte geogràfic amb què l’usuari ha interactuat.
        • idcategoria classifica aquest producte dins d’una categoria més àmplia.
        • epsg25831_x i epsg25831_y indiquen la ubicació del clic en el sistema EPSG:25831.
        • nivelzoom i ididioma permeten contextualitzar la interacció en funció del nivell de zoom i l'idioma de la interfície.
        '''

)

vn.train(
    documentation='''

        "taula_descargas":
        La taula descargas registra cada vegada que un usuari descarrega un arxiu cartogràfic del geoportal.
        • fechadescarga indica el moment en què es va realitzar la descàrrega.
        • idproducto i nomproducte identifiquen l’arxiu geoespacial descarregat.
        • format especifica el format del fitxer, com "GeoTIFF", "JPEG2000" o "SHP".
        • idcategoria ajuda a classificar els productes descarregats.
        • fulls permet associar la descàrrega amb una àrea geoespacial específica.
        • idepoca indica la temporalitat de la informació descarregada.
        '''

)

vn.train(
    documentation='''

       "taula_fulls":
        La taula fulls permet georeferenciar alguns dels productes descarregats.
        • idfull, nomproducte i geometria defineixen la relació espacial dels fulls amb les descàrregues.
        • Aquesta taula es relaciona amb descargas per facilitar la representació espacial de les descàrregues en mapes.
       '''

)

vn.train(
    documentation='''
        "relacions_i_anàlisi":
        Les relacions entre aquestes taules permeten una anàlisi completa del comportament dels usuaris al geoportal.
        • Creuar la informació de clics i descàrregues amb usuarios permet estudiar patrons d’ús segons àmbits professionals, idiomes o perfils d’usuari.
        • Relacionar els productes que els usuaris consulten amb aquells que descarreguen ajuda a avaluar l’efectivitat del sistema en la distribució d’informació.
        • L’anàlisi de fechadescarga i fecha en clics permet identificar tendències temporals en la demanda de dades cartogràfiques.
        • La connexió entre descargas i fulls facilita la geolocalització de descàrregues per generar mapes de demanda.
        '''

)



Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


'3251460-doc'

In [ ]:
vn.train(
    documentation='''
    "descargas massives":
        Un usuario realiza **descargas masivas** cuando efectúa más de **50 descargas en una ventana de 30 minutos**.
        Para detectar este comportamiento:
        1. Se cuentan las descargas de cada usuario en intervalos móviles de 30 minutos.
        2. Se identifican los eventos donde un usuario supera las 50 descargas en ese tiempo.
        3. Se cuentan los eventos de descargas masivas de cada usuario.
        4. Finalmente, se obtiene un listado de usuarios ordenados según el número de eventos de descargas masivas.

        Esta métrica permite identificar **usuarios que realizan descargas en grandes volúmenes en cortos períodos de tiempo**.,'''
)


Adding documentation....


'3251463-doc'

In [ ]:
vn.train(
    documentation='''
        "usuario recurrente o constante o descargas recurentes":
        Un **usuario recurrente o constante** es aquel que ha realizado descargas de manera **consistente todos los años desde que se tienen registros**.
        Para definir esto:
        1. Se extraen los años en los que existen descargas en la base de datos.
        2. Se analizan los usuarios y los años en los que han realizado descargas.
        3. Se identifican los usuarios que han descargado datos **cada año sin interrupciones**.
        4. Se filtran únicamente aquellos que han estado activos durante todos los años de la base de datos.

        Este criterio permite detectar **usuarios con un uso continuo de la plataforma a lo largo del tiempo**.
        ''')


Adding documentation....


'3251462-doc'

In [ ]:
vn.train(
    documentation='''
    "percentatge descàrregues dones":
        Per calcular el **percentatge de descàrregues fetes per dones** sobre el total de descàrregues, s'han d'excloure del total aquelles descàrregues on el gènere de l'usuari sigui **'No respost'** o **'No es recull'**.

        Procediment:
        1. Es compten les descàrregues realitzades per usuàries amb gènere = 'Dona'.
        2. S'obté el total de descàrregues, excloent aquelles on el gènere sigui 'No respost' o 'No es recull'.
        3. Es calcula el percentatge
        **Regla clau:**
        - Sempre que es facin consultes relacionades amb el gènere dels usuaris, el total de descàrregues ha de **excloure** aquelles on `usuaris.genere IN ('No respost', 'No es recull')`.

        Aquesta mètrica permet obtenir una representació més precisa de la participació de les dones en les descàrregues, evitant biaixos generats per dades desconegudes.
    '''
)


Adding documentation....


'3251461-doc'

Adding documentation....


'3251450-doc'

In [ ]:
# consultar los datos de entrenamiento
training_data = vn.get_training_data()
training_data

,id,training_data_type,question,content
0,1003400-sql,sql,En quin moment de l’any s’han realitzat més de...,\n WITH descargas_mensuales AS (\n ...
1,1003416-sql,sql,Indica les descàrregues totals de tots els pro...,"\n SELECT idcategoria, COUNT(*) AS tota..."
2,1003382-sql,sql,Quants usuaris no han fet cap descàrrega?,\n WITH total_usuarios AS (\n SELECT...
3,1003410-sql,sql,Crea un gràfic de línies on visualitzem l’evol...,\n SELECT\n DATE_TRUNC('mont...
4,3251456-doc,documentation,None,"\n ""taula_usuarios"":\n La taula ..."
5,1003380-sql,sql,Quants usuaris estan inactius?,\n SELECT COUNT(*) AS usuarios_inactivo...
6,1003417-sql,sql,Quin és el número de descàrregues totals duran...,\n SELECT COUNT(*) AS total_descargas_2...
7,1003369-sql,sql,Quin percentatge del total d’usuaris represent...,\n WITH total_filtrado AS (\n SELECT...
8,1003391-sql,sql,Quants usuaris nous es registren cada mes?,\n WITH registros_mensuales AS (\n ...
9,1003377-sql,sql,Existeix alguna relació entre idioma i format ...,\n WITH descargas_por_idioma_formato AS (\n...


In [ ]:
# revover algun dato de entrenamiento
vn.remove_training_data(id='1-ddl')
